# 📊 Evaluación rigurosa de modelos RNN: Perplejidad y palabras fuera de vocabulario

Este notebook presenta una implementación estructurada para evaluar modelos LSTM en tareas de modelado de lenguaje, 
haciendo especial énfasis en métricas como la *Perplejidad* (PP) y el manejo de palabras fuera de vocabulario (*Out-Of-Vocabulary*, OOV).

## 🔧 BLOQUE 1: Setup y descarga de datos

- Instalación de dependencias necesarias (`torch`, `torchtext`, `nltk`, etc.).
- Descarga del dataset **WikiText-2**.
- Tokenización del corpus.
- Construcción de vocabulario con diferentes tamaños:
  - 10,000 palabras
  - 30,000 palabras
  - 50,000 palabras

# 🔧 Bloque 1: Setup y descarga de datos

### 📚 Teoría
El dataset **WikiText-2** es un corpus ampliamente utilizado para tareas de modelado de lenguaje. Contiene texto derivado de artículos de Wikipedia y es útil para entrenar y evaluar modelos de predicción de texto.

Antes de entrenar cualquier modelo, es importante:
1. Descargar el corpus.
2. Tokenizar el texto.
3. Construir un vocabulario de tamaño controlado (ej. 10^4, 3x10^4, 5x10^4 tokens).
4. Dividir en datasets de entrenamiento, validación y prueba.

Utilizaremos `torchtext`, que facilita todo este proceso con utilidades listas para usar.




In [2]:

# Instalación de dependencias necesarias (si no están disponibles)
!pip install torch torchtext


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from datasets import load_dataset

# Cargar WikiText-2
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Ver muestras
print(dataset["train"][0])
print(dataset["validation"][0])
print(dataset["test"][0])


Generating validation split: 100%|███████████████████████████████████████| 3760/3760 [00:00<00:00, 442658.18 examples/s]

{'text': ''}
{'text': ''}
{'text': ''}


In [4]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

# Selección de tokenizer tipo 'basic_english'
tokenizer = get_tokenizer("basic_english")

# Carga de datasets
train_iter = WikiText2(split='train')
valid_iter = WikiText2(split='valid')
test_iter = WikiText2(split='test')

# Tokenización y conteo de frecuencias
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))

# Ejemplo: Crear vocabulario de tamaño 10^4
vocab_size = 10_000
vocab = Vocab(counter, max_size=vocab_size, specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HTTPError: 403 Client Error: Forbidden for url: https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
This exception is thrown by __iter__ of HTTPReaderIterDataPipe(skip_on_error=False, source_datapipe=OnDiskCacheHolderIterDataPipe, timeout=None)

In [5]:
# ---- BLOQUE 1: Setup y descarga de datos ----

# Instalación de librerías necesarias
#!pip install datasets -q

# Importar librerías
from datasets import load_dataset
import nltk
import torch
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter

# Descargar recursos de NLTK (tokenizador)
nltk.download('punkt')

# Descargar WikiText-2 usando HuggingFace
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenizar el texto
def tokenize(text):
    return nltk.word_tokenize(text)

# Preparar los datasets tokenizados
train_tokens = [tokenize(example['text']) for example in dataset['train']]
valid_tokens = [tokenize(example['text']) for example in dataset['validation']]
test_tokens  = [tokenize(example['text']) for example in dataset['test']]

# Construir vocabulario (ejemplo: 10k palabras más frecuentes)
def build_vocab(token_lists, vocab_size=10000):
    counter = Counter()
    for tokens in token_lists:
        counter.update(tokens)
    vocab = build_vocab_from_iterator([counter.keys()], specials=["<unk>"], max_tokens=vocab_size)
    vocab.set_default_index(vocab["<unk>"])
    return vocab

# Crear tres vocabularios de diferentes tamaños
vocab_10k = build_vocab(train_tokens, vocab_size=10_000)
vocab_30k = build_vocab(train_tokens, vocab_size=30_000)
vocab_50k = build_vocab(train_tokens, vocab_size=50_000)

print("Bloque 1 completado ✅")


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Generating validation split: 100%|███████████████████████████████████████| 3760/3760 [00:00<00:00, 317411.35 examples/s]


Bloque 1 completado ✅


## 🧠 BLOQUE 2: Definición del modelo

Se define un modelo LSTM con las siguientes características:

- Embeddings de dimensión 300.
- Dos capas LSTM con tamaño oculto de 512.
- Capa final densa con softmax para predecir la siguiente palabra.

**Componentes del modelo:**
- `Embedding layer`
- `LSTM layers (stacked)`
- `Linear output layer`

In [6]:
# ---- BLOQUE 2: Definición del modelo ----

import torch.nn as nn

class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=300, hidden_dim=512, num_layers=2):
        super(LSTMLanguageModel, self).__init__()
        
        # Capa de embeddings
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # LSTM de 2 capas
        self.lstm = nn.LSTM(
            input_size=embed_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers, 
            batch_first=True
        )
        
        # Capa final (proyección de hidden_dim a vocabulario)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        embeds = self.embedding(x)
        
        if hidden is None:
            output, hidden = self.lstm(embeds)
        else:
            output, hidden = self.lstm(embeds, hidden)
        
        logits = self.fc(output)
        return logits, hidden


In [7]:
# Crear un modelo de ejemplo
vocab_size = len(vocab_10k)  # usamos el vocabulario de 10k palabras por ahora
model = LSTMLanguageModel(vocab_size)

# Probar con un batch falso (batch_size=4, secuencia de 20 tokens)
x_dummy = torch.randint(0, vocab_size, (4, 20))  # random integers como tokens
logits, hidden = model(x_dummy)

print(f"Shape de salida logits: {logits.shape}")  # debería ser [4, 20, vocab_size]


Shape de salida logits: torch.Size([4, 20, 10000])


## 🏋️ BLOQUE 3: Entrenamiento

Entrenamos tres versiones del modelo, una por cada tamaño de vocabulario.

**Funciones incluidas:**
- Entrenamiento y validación por épocas.
- Registro de métricas:
  - *Perplejidad* sobre conjunto de validación.
  - Conteo y porcentaje de palabras OOV.

In [ ]:
# ---- BLOQUE 3: Entrenamiento ----

from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import math

# 1. Dataset personalizado para lotes de secuencias
class LanguageModelDataset(Dataset):
    def __init__(self, token_lists, vocab, seq_len=30):
        self.vocab = vocab
        self.seq_len = seq_len
        
        # Convertir todo en una gran lista de IDs
        self.data = [vocab[token] for tokens in token_lists for token in tokens if token.strip()]
        
    def __len__(self):
        return len(self.data) // self.seq_len

    def __getitem__(self, idx):
        start = idx * self.seq_len
        end = start + self.seq_len + 1
        chunk = self.data[start:end]

        # x son los primeros n tokens, y son los siguientes n tokens
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y

# 2. Funciones de entrenamiento y validación
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits, _ = model(x)
        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            logits, _ = model(x)
            loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            total_loss += loss.item()

    return total_loss / len(dataloader)

# 3. Entrenamiento para un vocabulario (ejemplo vocab_10k)

# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
seq_len = 30
num_epochs = 5
learning_rate = 0.001

# Crear datasets y dataloaders
train_dataset = LanguageModelDataset(train_tokens, vocab_10k, seq_len=seq_len)
valid_dataset = LanguageModelDataset(valid_tokens, vocab_10k, seq_len=seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# Instanciar modelo
model = LSTMLanguageModel(len(vocab_10k)).to(device)

# Optimizador y función de pérdida
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Entrenamiento
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    valid_loss = evaluate(model, valid_loader, criterion, device)
    perplexity = math.exp(valid_loss)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | Perplexity: {perplexity:.2f}")


## 🧩 BLOQUE 4: Manejo de OOV

Se comparan tres estrategias de tratamiento para palabras fuera de vocabulario:

1. **Token `<UNK>`:** se reemplazan todas las palabras desconocidas por un token especial.
2. **Modelo char-level:** backoff a un modelo de caracteres para predecir embeddings.
3. **Similitud de Levenshtein:** reemplazo por la palabra más cercana en el vocabulario.

## 📈 BLOQUE 5: Evaluación

Métricas medidas:

- **Perplejidad** en el conjunto de prueba.
- **Porcentaje de OOV** en test.
- Comparación entre modelos con distintos vocabularios y estrategias OOV.

## 📊 BLOQUE 6: Análisis y visualización

Incluye:

- Gráficos de perplejidad vs. tamaño de vocabulario.
- Cobertura del vocabulario vs. porcentaje de OOV.
- Comparativa de estrategias de manejo de OOV.

## 📘 BLOQUE 7: Reporte final

Resumen de hallazgos:

- Impacto del tamaño del vocabulario sobre la perplejidad.
- Eficacia relativa de cada estrategia de OOV.
- Recomendaciones para aplicaciones en producción.

**Exportable a PDF o documento académico.**